In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import ARDRegression
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
age = pd.read_csv('./data/age_gender_info.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [3]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [4]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i

for df in [train, test, age]:
    df['지역'] = df['지역'].map(local_map)

In [5]:
# 인구 정보
# drop_cols = list(age.columns)[1:]
# age['0세이상20세미만'] = age[['10대미만(여자)','10대미만(남자)','10대(여자)','10대(남자)']].sum(axis=1)
# age['20세이상60세미만'] = age[['20대(여자)','20대(남자)','30대(여자)','30대(남자)','40대(여자)','40대(남자)','50대(여자)','50대(남자)']].sum(axis=1)
# age['60세이상'] = age[['60대(여자)','60대(남자)','70대(여자)','70대(남자)','80대(여자)','80대(남자)','90대(여자)','90대(남자)','100대(여자)','100대(남자)']].sum(axis=1)
# age.drop(drop_cols, axis=1, inplace=True)

In [6]:
# gender, age 정보 통합
# train = train.merge(age, on='지역')
# test = test.merge(age, on='지역')

# 전용면적 범주화
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

# 가구당 할당 주차면수
train['가구당할당주차면수'] = train['단지내주차면수'] / train['총세대수']
test['가구당할당주차면수'] = test['단지내주차면수'] / test['총세대수']

# 공가율
train['공가율'] = train['공가수'] / train['총세대수']
test['공가율'] = test['공가수'] / test['총세대수']

# 임대보증금 & 임대료
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

# 지하철 & 버스
train[['지하철','버스']] = train[['지하철', '버스']].fillna(0)
test[['지하철','버스']] = test[['지하철','버스']].fillna(0)

# 신분
test.loc[(test.신분.isnull()) & (test.단지코드 == "C2411"), '신분'] = 'A'
test.loc[(test.신분.isnull()) & (test.단지코드 == "C2253"), '신분'] = 'C'

# 공급유형
train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

In [7]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스', '가구당할당주차면수', '공가율']
#           '0세이상20세미만', '20세이상60세미만', '60세이상']
target = '등록차량수'

area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')
     
quali_columns = []
for ql in train['신분'].unique():
    quali_columns.append(f'{ql}_가구수')
    
type_columns = []
for tp in train['공급유형'].unique():
    type_columns.append(f'{tp}_가구수')

In [8]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [9]:
# 단지코드 별 통합

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    for col in quali_columns:
        qual = col.split('_')[0]
        new_train.loc[i, col] = temp[temp['신분']==qual]['전용면적별세대수'].sum()
        
    for col in type_columns:
        tp = col.split('_')[0]
        new_train.loc[i, col] = temp[temp['공급유형']==tp]['전용면적별세대수'].sum()
        
    new_train.loc[i,'총임대가구수'] = temp['전용면적별세대수'].sum()
    new_train.loc[i,'임대보증금(가중)'] = int(((temp['전용면적별세대수']/new_train.loc[i,'총임대가구수'])*temp['임대보증금']).sum())
    new_train.loc[i,'임대료(가중)'] = int(((temp['전용면적별세대수']/new_train.loc[i,'총임대가구수'])*temp['임대료']).sum())
    new_train.loc[i, target] = temp.loc[0, target]
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
    for col in quali_columns:
        qual = col.split('_')[0]
        new_test.loc[i, col] = temp[temp['신분']==qual]['전용면적별세대수'].sum()
        
    for col in type_columns:
        tp = col.split('_')[0]
        new_test.loc[i, col] = temp[temp['공급유형']==tp]['전용면적별세대수'].sum()
        
    new_test.loc[i,'총임대가구수'] = temp['전용면적별세대수'].sum()
    new_test.loc[i,'임대보증금(가중)'] = int(((temp['전용면적별세대수']/new_test.loc[i,'총임대가구수'])*temp['임대보증금']).sum())
    new_test.loc[i,'임대료(가중)'] = int(((temp['전용면적별세대수']/new_test.loc[i,'총임대가구수'])*temp['임대료']).sum())

423it [00:15, 27.35it/s]
150it [00:05, 26.78it/s]


In [10]:
new_train['임대율'] = new_train['총임대가구수']/new_train['총세대수']
new_test['임대율'] = new_test['총임대가구수']/new_test['총세대수']

In [11]:
x_train = new_train.drop(['단지코드',target], axis=1)
y_train = new_train[target]
x_test = new_test.drop(['단지코드'], axis=1)

In [12]:
ard = ARDRegression()
adb = AdaBoostRegressor()
rf = RandomForestRegressor()
etr = ExtraTreesRegressor()

In [13]:
folds= 5
kf = KFold(n_splits=folds, shuffle=True, random_state=42)

In [14]:
models = [ard, adb, rf, etr]
names = ['ard', 'adb', 'rf', 'etr']
maes = []
mapes = []
for model in models:
    local_mae = 0
    local_mape = 0
    
    for i, (tr_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
        X_tr, X_val = x_train.iloc[tr_idx,:], x_train.iloc[val_idx,:]
        y_tr, y_val = y_train[tr_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        
        mae = mean_absolute_error(model.predict(X_val), y_val)
        mape = mean_absolute_percentage_error(model.predict(X_val), y_val)
        
        local_mae+= mae/folds
        local_mape+= mape/folds
    
    maes.append(round(local_mae,4))
    mapes.append(round(local_mape,4))
        
for i, j, model in zip(maes, mapes, names):
    print('\n',model)
    print('mae', i)
    print('mape', j)


 ard
mae 125.0825
mape 0.2774

 adb
mae 158.8551
mape 0.299

 rf
mae 129.8984
mape 0.2453

 etr
mae 127.2994
mape 0.237


In [15]:
pred = (rf.predict(x_test) + etr.predict(x_test))/2

In [17]:
submission['num'] = pred
submission.to_csv('submission/rf+etr.csv', index=False)